### 深層学習（day4)　レポート

- #### Section1 強化学習
    - 長期的に報酬を最大化できるように環境のなかで行動を選択できるエージェントを作ることを目標とする機械学習の１分野
    - 行動の結果として与えられる利益（報酬）をもとに、行動を決定する原理を改善していく仕組み
    - 探索と利用のトレードオフ
        -　過去のデータでベストとされる行動のみを取り続ければ、他のより良い行動を見つけることはできない（探索が足りない状態）。一方で、未知の行動のみを常に取り続ければ、過去の経験を活かすことはできない（利用が足りない状態）。
    - 強化学習と、教師あり・なし学習の違い
        - 結論として、目標が異なる。
        - 教師あり・なし学習ではデータに含まれるパターンの発見や、そのデータからの予測が目標だった
        - 強化学習では、優れた方策を見つけることが目標である
    - 価値関数
        - 価値を表す関数としては、状態価値関数と行動価値関数の２種類がある
        - ある状態の価値に注目する場合は状態価値関数
        - 状態と価値を組み合わせた価値に注目する場合は行動価値関数
    - 方策関数
        - 方策ベースの強化学習手法において、ある状態でどのような行動を採るのかの確率を与える関数
    - 方策勾配法
        - 方策をモデル化して最適化する手法
        - $ \theta^{(t+1)} = \theta^{(t)} + \epsilon \nabla J(\theta) $
        - ここでJは方策の良さを示すもの。定義する必要あり
            - 定義方法
                - 平均報酬
                - 割引報酬和
            - 上記定義に対応して行動価値関数Q(s,a)の定義を行い、方策勾配定理が成り立つ.
            $ \nabla_{\theta}J(theta) = E_{\pi_{\theta}}[( \nabla_{\theta}log\pi_{\theta}(a|s)Q^{\pi}(s,a))]   $
        
- #### Section2 AlphaGo
    - 囲碁のコンピュータ・プログラム。囲碁は盤面のパターン数が膨大であり、他のゲームと比較してコンピュータが人間に勝つのは困難だと考えられていたが、プロ棋士等にも勝利したことで話題になった。それまでのAIとは異なり、NNを応用。
    - 方策関数にPolicy Net,価値関数にValue Netという畳み込みNNを利用。前者では19×19マスの着手予想確率、後者では現局面での勝率を-1~1で表したものが出力される。
    - RolloutPolicy
        - NNではなく線形の方策関数。探索中に高速に着手確率を出すために使用。出力は各マスの着手予想確率
    - 学習ステップ
        1. 教師あり学習によるRolloutPolicyとPolicyNetの学習
        2. 強化学習によるPolicyNetの学習
        3. 強化学習によるValueNetの学習
    - モンテカルロ木探索
        - コンピュータ囲碁ソフトでは現在もっとも有効とされている探索法
        - 他のボードゲームではminmax探索やその派生を使うことが多いが、それには盤面価値や勝率予想値が必要となり、囲碁ではそれらを出すのが困難とされてきた
        - そこで、盤面評価値に頼らず末端評価値（つまり勝敗のみ）を使って探索できないかという発送で生まれたのが本手法。囲碁の場合は、他のb−ドゲームと違い最大手数はマス数でほぼ固定されるため末端局面に到達しやすい。
        - 現局面から末端局面までPlayOutと呼ばれるランダムシミュレーションを多数回行い、その勝敗を集計して着手の優劣を決定する。また、該当手のシミュレーション回数が一定数を超えたらその手を着手した後の局面を開始局面とするように探索木を成長させる。（これがモンテカルロ木探索の優れているところ）
    - Alpha Go Zero ( vs Alpha Go(Lee))
        - 教師あり学習を一切行わず、強化学習のみで作成
        - 特徴入力からヒューリスティックな要素を排除。石の位置のみとした
        - PolicyNetとValueNetを１つのネットワークに統合
        - ResidualNetを導入
        - モンテカルロ木探索からRolloutシミュレーションをなくした
    - Residual Network
        - NWにショートカット構造を追加して、勾配の爆発・消失を抑える降下を狙ったもの。本構造の使用により、100層を超えるNWでの安定した学習が可能となった。
        - 基本構造は、 Conv -> BatchNorm -> ReLU -> Conv -> BatchNorm -> Add ReLUのブロックを1単位として積み重ねる形となる
        - Residual Networkを使うことにより、層数の違うNWのアンサンブル効果が得られているという説もある
        
    
- #### Section3 軽量化・高速化技術
    - 分散深層学習
        - 深層学習は多くのデータを使用したり、パラメタ調整のために多くの時間を使用したりするため、高速な計算が求められる。そのため、複数の計算資源（ワーカー）を使用し並列的にNNを構成することで、効率よく学習を行いたい
        -　　モデルが大きいときはモデル並列化を、データが大きいときはデータ並列化をすると良い
        - データ並列化
            - 親モデルを各ワーカーに子モデルとしてコピー
            - データを分割し、各ワーカーごとに計算させる
            - 各モデルのパラメタの合わせ方で、同期型か非同期型に分かれる
                - 同期型
                    - 各ワーカーが計算が終わるのを待ち、勾配の平均を計算し、親モデルのパラメタを更新
                    - 非同期型に比べて処理速度は下がるが、比較的に学習が安定的であり、また精度が良いことが多いので主流となっている
                - 非同期型
                    - 各ワーカーはお互いの計算を待たずに各子モデル毎に更新を行う。学習が終わった子も出るはパラメタサーバにpushされ、新たに学習を始めるときは、パラメタサーバからpopしたモデルに対して学習していく
                    - 計算を待たないため、処理速度は非同期型の方が早い
                    - 非同期型は最新のモデルのパラメタを使用できないため、学習が不安定になりやすい
        - モデル並列化
            - 親モデルを各ワーカーに分割し、それぞれのモデルを学習させる、全てのデータで学習が完了後、１つのモデルに復元
            - モデルのパラメタ数が多いほど、スピードアップの効率も上昇
        - GPUによる高速技術
            - GPGPU(General-purpose on GPU) グラフィック以外の用途で使用されるGPUの総称
            - GPUはCPUと比較して簡単な並列処理が得意。NNの学習は単純な行列演算が多いので、GPUを用いた高速化が可能
            - GPGPU開発環境
                - Deep Learningフレームワーク（Tensorflow,Pytorch)内で実装されているので、使用する際は指定すれば良い
                - CUDA
                    - NVDIAが開発しているGPUのみで使用可能。DL用に提供されているので使いやすい
                - OpenCL 
                    - openな並列コンピューティングPF.NVDIA以外のGPUからでも使用可能。DL用の計算に特化しているわけではない
    - モデルの軽量化
        - modelの軽量化はモバイル、IoT機器において有用（両者はPCと比して計算速度やメモリで劣る）
        - 軽量化の代表的な手法
            - 量子化（Quantization）
                - 通常のパラメタの64bit浮動小数点を32bitなど下位の精度に落とす
                - 計算の高速化と省メモリ化が期待できる
                - 一方で精度は多かれ少なかれ下がる。量子化する際は極端に精度が落ちないようにしなければならない
            - 蒸留１（Distillation）
                - 規模の大きなモデルの知識を使い、軽量なモデルの作成を行う
                - 言い換えると、学習済の精度の高いモデルの知識を、軽量なモデルに継承することを狙う
                - 蒸留は、教師モデルと生徒モデルの２つで構成される。教師モデルの重みを固定し生徒モデルの重みを更新していく。誤差は教師モデルと生徒モデルのそれぞれの誤差を使い重みを更新していく。
                    - 教師モデル
                        - 予測精度の高い、複雑なモデルやアンサンブルされたモデル
                    - 生徒モデル
                        - 教師モデルをもとに作られる軽量なモデル
            - プルーニング
                - modelの性能に寄与が少ないニューロンを削減することでモデルを圧縮し、高速化と軽量化を見込む
                - 重みがしきい値以下の場合にニューロンを削減し、再学習を行う手法
                - しきい値を大きくするとニューロンも大きく削減できるが、精度も減少する
    
- #### Section4 応用モデル
    - MobileNet
        - Depthwise-ConvolutionとPointwise-Convolutionにより、軽量化を実現。
        - 通常のCNNでは、空間方向とチャネル方向の畳み込みを一度に行っており、計算量が大きくなる（全ての要素が乗算）が、MobileNetでは空間方向とチャネル方向の畳込みの２段階に分けることで、軽量化を実現した。
        - Depthwise-Convolution
            - 仕組み
                - 入力マップのチャネルごとに畳み込みを実施
                - 出力マップをそれらと結合（入力マップのチャネル数と同じになる）
            - 通常の畳み込みカーネルは全ての層にかかっていることを考えると計算量が大幅減
                - $ HWCK^2 $
                - (一般的な畳み込みの場合は） $ HWCK^2M $
            - 各層ごとの畳込みなので層間の関係性は全く考慮されない。通常はPW畳み込みとセットで使うことで解決
        - Pointwise-Convolution
            - 仕組み
                - 1×1convとも呼ばれる（正確には１×１×ｃ）
                - 入力マップのポイントごとに畳み込みを実施
                - 出力マップ（チャネル数）はフォルタ数分だけ作成可能（任意のサイズが指定可能）
            - 出力マップの計算量
                - $ HWCM $
    - DenseNet
        - CNNアーキテクチャの一種。より深い層の実現を目指したもの。ResNetを踏まえ開発された。
        - ResNetではshortcut-connectionが導入されたが、DenseNetでは、前方の各層からの全ての出力が後方の層へ入力されているような密な層間の結合を有している、DenseBlockと呼ばれる構造を持つアーキテクチャ
        - DenseBlockでは成長率（Growth Rate)と呼ばれるハイパラが存在
            - DenseBlock内の各ブロック毎にk個ずつ特徴マップのチャネル数が増加していくとき、そのkを成長率と呼ぶ
        
    - WaveNet
        - 生の音声波形を生成する深層学習モデルであり、Pixel CNNを音声に応用したもの
            - Pixel CNN…高解像度の画像を精密に生成できる手法
        - WaveNetのメインアイディア
            - 時系列データに対して畳み込み（Dilated　Convolution）を適用する
            - Dilated Convolution
                - 層が深くなるにつれて、畳み込むリンクを離す
                - 受容野を簡単に深くすることができるという利点がある
    
- #### Section５ Transformer
    - ニューラル機械翻訳の問題点:長さに弱い
        - 文長が長くなると、表現力が足りなくなる
    - Attention(注意機構）
        - 翻訳先の各単語を選択する際に、翻訳元の文中の各単語の隠れ状態を利用
    - Attentionは何をしている？
        - Attentionは辞書オブジェクト
            - query(検索クエリ）に一致するkeyを索引し、対応するvalueを取り出す操作とみなすことができる
    - Transformer
        - 2017年6月に登場
        - RNNを使わない（必要なのはAttentionだけ）　　Attention　is all you need
        - 当時のSOTAをはるかに少ない計算量で実現した
        - RNNを使わない＝順序情報を持たないため、Positional　Encodingで単語の位置情報を付与する必要あり
        - 肝はMulti-head Attention
            - 重みパラメタの異なる8個のヘッドを使用
                - 8個のScaled Dot-Product Attentionの出力をconcat
                - それぞれのヘッドが異なる種類の情報を収集する
                    - Scaled Dot-Product Attention
                        - 全単語に関するAttentionをまとめて計算する
                          $ Attention(Q,K,V) = softmax(\frac{QK^T}{\sqrt{d_{k}}})V $
                          
- #### Section6 物体検知・セグメンテーション
    - 物体認識タスクの種類（下に行くほどタスク難易度高）
        - 分類：画像に対して、単一あるいは複数のクラスラベルを付与
        - 物体検知：バウンディングボックスで物体を検知
        - 意味領域分割：各ピクセルに対して単一のクラスラベル
        - 個体領域分割：各ピクセルに対して単一のクラスラベル
    - 物体認識のための有名なデータセットは様々なものがある（ILSVRC17など）が、データセットを用いる際には、目的に応じたBox/画像（ratio)のセットを用いた方がよい
        - Box/画像が小さいということは、アイコン的な映りであり、日常感とはかけ離れやすい
    - 評価指標
        - 分類問題での評価指標の復習
            - 混同行列→Accuracy,Precision,Recall,
        - IoU(Intersection over Union)
            - Motivation : 物体検出においては、クラスだけでなく物体位置の予測精度も評価したい
            - $ IoU = \frac{Area Of Overlap}{Area Of Union}$
            - Confusion Matrixの要素を用いて表現
                - $ IoU = \frac{TP}{TP+FP+FN} $
                - 別名Jaccard係数とも呼ばれる
        - mAP(mean Average Precision)
            - AP Average Precision
                - $ AP = \int_0^1P(R)dR $
                - PR曲線の下側面積
            - クラス数がCのとき、
                - $ mAP = \frac{1}{C}\Sigma_{i=1}^C AP_i $
        - FPS(Flames per Second)
            - 応用上の要請から、検出精度に加え、検出速度も問題となる
    - 物体検知の大枠
        - 物体検知のフレームワークは大きく２種類
            - 2段階検出器（Two-stage detector）
                - 候補領域の検出とクラス推定を別々に行う
                - 相対的に精度が高い傾向
                - 相対的に計算量が大きく推論も遅い傾向
            - １段階検出器(One-stage detector)
                - 候補領域の検出とクラス推定を同時に行う
                - 相対的に精度が低い傾向
                - 相対的に計算量が小さく推論も早い傾向
        - SSD
            - 1段階検出器の１つ
            - YOLOと並んで有名なアーキテクチャ
            - DefaultBoxを用意し、DefaultBoxを検出したい物体に合わせて変形させ、Confを出力する
            - 多数のDefaultBoxを用意したことで生ずる問題と対策
                - Non-Maximum Suppression
                    - IoUを算出し、一定以上であればconfが最も高いBoxのみを残す
                - Hard Negative Mining
                    - Negative はPositiveに対してどの程度まで、というような制約条件を課す
        - Semantic Segmentation
            - Semantic　Segmentationでは、ピクセル毎に意味を持たせるので、解像度が重要、
            - だが、Convolution・Poolingでは解像度が落ちる。→如何に戻すか
            - upsampling
                - 解像度を元に戻すことをupsamplingという
            - Deconvolution/Transposed convolution
                - 特徴マップのピクセル感覚をstrideだけ空け、特徴マップのまわりに(kernel size - 1) - paddingだけ余白を作り、畳込み演算を行う
                - 例えば、kernel size=3,padding=1,stride=1のDeconv.により、3×３の特徴マップは5×５にUp-samplingされる
                - 逆畳み込みと呼ばれることも多いが、畳込みの逆演算ではないことに注意。当然、poolingで失われた情報が復元されるわけではない

- #### SectionEX DCGAN
    - GAN(Generative Adversarial Nets)とは
        - 生成器と識別器を競わせて学習する生成＆識別モデル
            - Generator:乱数からデータを生成
            - Discriminator:入力データが真データ（学習データ）であるかを識別
            - GeneratorはDiscriminatorに誤判断させたい、Disciriminatorは正しく判別したい　→２プレイヤーのミニマックスゲーム
        - 2playerのミニマックスゲーム
            - 一人が自分の勝利する確率を最大化する作戦をとる
            - もうひとりは相手が勝利する確率を最小化する作戦をとる
            - GANでは価値関数V（バイナリークロスエントロピー）に対して、Dが最大化、Gが最小化を行う
    - DCGAN(Deep Convolutional GAN)とは
        - GANを利用した画像生成モデル
        - いくつかの構造制約により生成品質を向上
            - Generator
                - Pooling層の代わりに転置畳み込み層を使用
                    - 転置畳み込み層により乱数を画像にアップサンプリング
                - 最終層はtanh、その他はReLUで活性化
            - Discriminator
                - Pooling層の代わりに畳み込み層を使用
                    - 畳み込み層により画像から特徴量を抽出し、最終層をsigmoidで活性化
                - Leaky ReLUで活性化
            - 共通事項
                - 中間層に全結合層を使わない
                - BNを使用
        
